Загружаем необходимые библиотеки

In [93]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, min, count
from pyspark.sql.functions import col, mean
from pyspark.sql.functions import col, mean, count, expr, sum as spark_sum, when

# **Задание 1**

Создаем SparkSession

In [94]:
spark = SparkSession.builder \
    .appName("JSON to DataFrame") \
    .getOrCreate()

Считываем файл 1.json и создаем Spark DataFrame

In [95]:
df = spark.read.json("/content/1.json")

Выводим первые 12 строк

In [96]:
df.show(12)

+-----------------+--------------------+---------------+--------------------+-----+----------+---------------+--------+------+--------------------+
|         category|         description|expiration_date|                name|price|product_id|production_date|quantity|rating|              status|
+-----------------+--------------------+---------------+--------------------+-----+----------+---------------+--------+------+--------------------+
|           Фрукты|Прошел сертификац...|     2025-03-09|              Яблоко| 2241|         1|     2013-08-18|      56|  1.41|           Delivered|
|             Мясо|Отличается высоки...|     2002-01-15|      Куриные грудки| 9467|         2|     1998-09-04|     472|  0.21|                 New|
|Молочные продукты|Способствует сниж...|     2020-05-01|               Шубат| 3812|         3|     2015-05-14|      11|  1.89|Available for Pickup|
|Молочные продукты|Прошел строгий ко...|     2023-03-03|            Пармезан| 9478|         4|     2018-03-02|  

 Выводим количество колонок

In [97]:
column_count = len(df.columns)
print(f"Количество колонок: {column_count}")

Количество колонок: 10


Выводим статистические характеристики для всех колонок

In [98]:
df.describe().show()

+-------+--------+--------------------+---------------+----+-----------------+------------------+---------------+------------------+------------------+--------------------+
|summary|category|         description|expiration_date|name|            price|        product_id|production_date|          quantity|            rating|              status|
+-------+--------+--------------------+---------------+----+-----------------+------------------+---------------+------------------+------------------+--------------------+
|  count|     791|                 791|            791| 791|              791|               791|            791|               791|               791|                 791|
|   mean|    NULL|                NULL|           NULL|NULL| 5162.02402022756|             396.0|           NULL| 507.6321112515803|2.5377117572692813|                NULL|
| stddev|    NULL|                NULL|           NULL|NULL|2955.231614762311|228.48632344190756|           NULL|286.43209800060936|1.4

Удаляем указанную колонку: description

In [99]:
df = df.drop("description")

Создаем новую колонку: цена в $ (price * (94.24

In [100]:
df = df.withColumn("price_in_$", df["price"] * 94.24)

Проверяем колонки

In [101]:
df.show(12)

+-----------------+---------------+--------------------+-----+----------+---------------+--------+------+--------------------+------------------+
|         category|expiration_date|                name|price|product_id|production_date|quantity|rating|              status|        price_in_$|
+-----------------+---------------+--------------------+-----+----------+---------------+--------+------+--------------------+------------------+
|           Фрукты|     2025-03-09|              Яблоко| 2241|         1|     2013-08-18|      56|  1.41|           Delivered|         211191.84|
|             Мясо|     2002-01-15|      Куриные грудки| 9467|         2|     1998-09-04|     472|  0.21|                 New|         892170.08|
|Молочные продукты|     2020-05-01|               Шубат| 3812|         3|     2015-05-14|      11|  1.89|Available for Pickup|         359242.88|
|Молочные продукты|     2023-03-03|            Пармезан| 9478|         4|     2018-03-02|     364|   0.1|            Shippin

Записываем полученный набор данных в файл: r_1.csv

In [102]:
df.write.csv("r_1.csv", header=True, mode='overwrite')

In [103]:
spark.stop() #Завершаем работу с Spark

# **Задание 2**

In [104]:
spark = SparkSession.builder \
    .appName("JSON to DataFrame") \
    .getOrCreate()

Считываем файл 1.json и создаем Spark DataFrame

In [105]:
df = spark.read.json("/content/2.json")

Выводим последние 5 строк

In [106]:
df.tail(5)

[Row(category='Electronics', description='Item 904', item_id=904, length=4888.74, price=1734.71, weight=6099.33, width=7732.28),
 Row(category='Musical Instruments', description='Item 905', item_id=905, length=270.35, price=2648.39, weight=2266.65, width=9327.94),
 Row(category='Art Supplies', description='Item 906', item_id=906, length=334.56, price=7888.39, weight=1222.35, width=89.4),
 Row(category='Musical Instruments', description='Item 907', item_id=907, length=2013.71, price=8507.64, weight=984.87, width=574.17),
 Row(category='Art Supplies', description='Item 908', item_id=908, length=487.62, price=74.2, weight=9007.79, width=1619.84)]

Отфильтруем данные, где length < 4622.0

In [107]:
filtered_df = df.filter(df["length"] < 4622.0)

Подсчитаем, сколько строк было отфильтровано на предыдущем шаге

In [108]:
filtered_count = filtered_df.count()
print(f"Количество отфильтрованных строк: {filtered_count}")

Количество отфильтрованных строк: 730


Выполняем группировку по полю category, для каждой категории рассчитав: количество элементов в группе, avg price, min width, min length

In [109]:
grouped_df = filtered_df.groupBy("category").agg(
    count("*").alias("count"),
    avg("price").alias("avg_price"),
    min("width").alias("min_width"),
    min("length").alias("min_length")
)


Записываем полученный набор данных в файл: r_2.json

In [110]:
grouped_df.write.json("r_2.json", mode='overwrite')

In [111]:
spark.stop()

# **Задание 3**

In [112]:
from pyspark.sql.functions import col, mean, count, expr, sum as spark_sum, when

In [113]:
spark = SparkSession.builder \
    .appName("CSV to DataFrame") \
    .getOrCreate()

Считываем файл 3.csv и создаем Spark DataFrame.

In [114]:
df = spark.read.csv("/content/3.csv", header=True, inferSchema=True)

Считаем количество пропусков

In [115]:
missing_counts = df.select([(count(col(c).isNull()).alias(c)) for c in df.columns])
missing_counts.show()

+---+----+-----+---+----+---+---+----+-----+-----+------+----+------+------+---+----+------+---+----+------+------+---+---+----+---+----+---+---+----+---+----+---+---+----+---+----+----+---+---+---+---+---+---+---+---+---+----+----+---+------+----+----+---+----+
|785|-690|null2|923|-344|638|651|-710|-5168|null9|-56810|-723|null12|null13|762|-405|-51616|949|-632|null19|null20|310|784|-927|575|-403|341|-22|-272|239|-365|401|529|-108|694|-262|-388|886|394| 69|563|252| 90| -2|287|-57|-270|-876|156|-56849|-244|-569|737|-867|
+---+----+-----+---+----+---+---+----+-----+-----+------+----+------+------+---+----+------+---+----+------+------+---+---+----+---+----+---+---+----+---+----+---+---+----+---+----+----+---+---+---+---+---+---+---+---+---+----+----+---+------+----+----+---+----+
|289| 289|  289|289| 289|289|289| 289|  289|  289|   289| 289|   289|   289|289| 289|   289|289| 289|   289|   289|289|289| 289|289| 289|289|289| 289|289| 289|289|289| 289|289| 289| 289|289|289|289|289|289|289|2

Обработываем пропуски: заполняем средним по колонке

In [116]:
for column in df.columns:
    mean_value = df.select(mean(col(column))).first()[0]
    df = df.fillna({column: mean_value})

Создаем новый столбец с количеством положительных значений

In [117]:
positive_count_column = sum(when(col(c) > 0, 1).otherwise(0) for c in df.columns)
df = df.withColumn("positive_count", positive_count_column)

Производим фильтрацию данных: оставляем строки, в которых количество положительных элементов больше 0.

In [118]:
df_filtered = df.where(col("positive_count") > 0)

Подсчитываем статистику по каждой колонке и сохраняем в отдельный df

In [119]:
stats_df = df_filtered.describe()

Записываем полученный df со статистикой: r_3.csv

In [120]:
stats_df.write.csv("r_3.csv", header=True, mode='overwrite')

In [121]:
spark.stop()